## Использование SpanMarker с spaCy

Рассмотрим использование предобученных моделей SpanMarker с spaCy.

### Настройка
Прежде всего, необходимо установить модули Python `spacy` и `span_marker`. После этого нам также потребуется установить модель `spacy`. Выберем самую простую на данный момент: `en_core_web_sm`

In [1]:
# %pip install span_marker spacy
# !spacy download en_core_web_sm

### Использование spaCy для распознавания именованных сущностей
Начнем с использования чистого spaCy для NER, чтобы получить представление об изменениях, которые необходимо внести для использования моделей SpanMarker вместо этого.

In [ ]:
import spacy

# Загружаем модель spaCy
nlp = spacy.load("en_core_web_sm")

# Подаем текст на вход модели, чтобы получить объект Doc от spacy
text = """Cleopatra VII, also known as Cleopatra the Great, was the last active ruler of the \
Ptolemaic Kingdom of Egypt. She was born in 69 BCE and ruled Egypt from 51 BCE until her \
death in 30 BCE."""
doc = nlp(text)

# И смотрим на сущности
doc.ents

Модуль `spaCy` поставляется с удобным визуализатором, который мы можем использовать для более наглядного просмотра этих сущностей. Давайте воспользуемся им.

In [3]:
from spacy import displacy

displacy.render(doc, style="ent")

Не идеально. Эта модель spaCy пропускает `Cleopatra VII`, считает `Cleopatra the Great` произведением искусства, а все даты принимает за числа и организации.

### Использование моделей обученных в SpanMarker для распознавания именованных сущностей с spaCy
Мы можем легко добавить модель SpanMarker как готовую замену оригинального конвейера NER от spaCy. Это делается так:

In [4]:
nlp.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-roberta-large-ontonotes5"})

Параметр `model` в конфигурации указывает на [tomaarsen/span-marker-roberta-large-ontonotes5](https://huggingface.co/tomaarsen/span-marker-roberta-large-ontonotes5), модель, обученную на OntoNotes v5.0, том же наборе данных, который используется в оригинальном конвейере NER от spaCy. В [справочнике по API интеграции с spaCy](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.spacy_integration.html) есть больше документации о параметрах конфигурации. Давайте опробуем обновленный конвейер spaCy.

In [5]:
# All we have to do is process the text using the updated spaCy pipeline
doc = nlp(text)

print(doc.ents)

displacy.render(doc, style="ent")

Но что, если мы не хотим использовать модель с такими метками? Эта интеграция работает с любой [моделью SpanMarker в хабе Hugging Face](https://huggingface.co/models?library=span-marker), так что мы можем просто выбрать другую. Давайте теперь также убедимся, что модель остается на CPU, просто чтобы посмотреть, как это работает. Кроме того, мы перезапишем сущности из собственной NER-модели spaCy. Это рекомендуется, когда модель SpanMarker использует другую схему меток, чем spaCy (которая использует метки из OntoNotes v5).

In [6]:
nlp.remove_pipe("span_marker")
nlp.add_pipe(
    "span_marker",
    config={
        "model": "tomaarsen/span-marker-xlm-roberta-base-fewnerd-fine-super",
        "device": "cpu",
        "overwrite_entities": True,
    },
)

doc = nlp(text)
print(doc.ents)
displacy.render(doc, style="ent")


### Итог
Использовать spacy с span_marker просто

In [7]:
import spacy

nlp = spacy.load("en_core_web_sm", exclude=["ner"])
nlp.add_pipe("span_marker")

text = "Amelia Earhart flew her single engine Lockheed Vega 5B across the Atlantic to Paris."
doc = nlp(text)

[(entity, entity.label_) for entity in doc.ents]